<a href="https://colab.research.google.com/github/toanpt74/COLAB_RD/blob/main/Question_Answer_Whith_TextStream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pyvi
from datetime import datetime
import os
import sys
import chromadb
import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
    PeftModel,
    PeftConfig,
    prepare_model_for_kbit_training
)

from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TextIteratorStreamer
)
from chromadb.utils import embedding_functions
from threading import Thread

os.environ["WANDB_MODE"] ="offline"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
base_model=r'D:\Tuan\ChatGPT\model\Vistral-7B-Chat'
device = "auto"
model_id = r"D:\Tuan\ChatGPT\model\phobert-base-v2"
embedding_fun = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_id)

quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16
    )

tokenizer = AutoTokenizer.from_pretrained(base_model,trust_remote_code=True, local_files_only=True)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map=device,
    quantization_config=quantization_config,
    local_files_only=True
)


system_prompt = "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể, đồng thời giữ an toàn.\n"
system_prompt += "Câu trả lời của bạn không nên chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Hãy đảm bảo rằng các câu trả lời của bạn không có thiên kiến xã hội và mang tính tích cực."
system_prompt += "Nếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác. Nếu bạn không biết câu trả lời cho một câu hỏi, hãy trẳ lời là bạn không biết và vui lòng không chia sẻ thông tin sai lệch.\n"

system_prompt+="Noi dung o day"


question ="Ai là CEO của VinGroup?"
conversation = [{"role": "system", "content": system_prompt}]
conversation.append({"role": "user", "content": f"{question}"})

input_ids = tokenizer.apply_chat_template(conversation, return_tensors="pt")
streamer = TextIteratorStreamer(tokenizer)

inputs = tokenizer.apply_chat_template(
    conversation,
    add_generation_prompt=True,
    return_tensors='pt' ).to(model.device)

Thread(target=lambda: model.generate(inputs, max_new_tokens=256, streamer=streamer)).start()

for text in streamer:
    print(text, end='', flush=True)


# question="Hồ Chí Minh là ai?"
# inputs = tokenizer(question, return_tensors="pt")
# streamer = TextIteratorStreamer(tokenizer)
#
# generation_kwargs = dict(
#         inputs, streamer=streamer,
#         max_new_tokens=1000,
#         do_sample=True,
#         top_p=0.95,
#         top_k=40,
#         temperature=0.1,
#         repetition_penalty=1.05,
#     )
#
# thread = Thread(target=model.generate, kwargs=generation_kwargs)
# thread.start()
#
#
# for new_text in streamer:
#   print(new_text, end="")
#
# thread.join() # join our thread